In [1]:
from dotenv import load_dotenv
load_dotenv()

from langchain_community.graphs import Neo4jGraph

# Warning Control
import warnings
warnings.filterwarnings("ignore")

In [ ]:
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE')